In [1]:
# imports
from ema_workbench.analysis import parcoords
import pandas as pd
import numpy as np
from ema_workbench import load_results
import seaborn as sns
import os
from model.enums import BeliefUpdate
from dmdu.utils_dmdu import get_lever_names, get_outcome_names

sns.set(rc={'figure.figsize': (14, 10)})

# Overall Plan:
1. avg over replications
2. non-dominated sort -> Pareto-optimal policies (& their outcomes)
3. parallel axis plots of levers & outcomes (to inspect Pareto-optimal policies)


### 1. Loading the data (results), avg over replications, save into .csv
(can just avg bc replications to make sure that sensitivity to stochasticity is not an issue wrt reproducibility)

In [2]:
# Params  # TODO: make a function 'load_my_csv(nfe, bufn, filename, seed=None)' make it also remove the "Unnamed: 0" col
n_repl = 30
bufns = list(BeliefUpdate)  # [BeliefUpdate.DEFFUANT]  # -> BeliefUpdate.list

for bufn in bufns:

    # Loading the results
    filename = "full_factorial" + f"_{n_repl}_replications" + f"_{bufn.name}"
    results_path = os.path.join(os.getcwd(), "data", filename)
    experiments, outcomes = load_results(results_path)

    # # Avg over replications
    # avgs_outcomes = {}
    # for metric in outcomes.keys():
    #     avgs = np.mean(outcomes[metric], axis=1)
    #     avgs_outcomes[metric] = avgs
    #
    # # Save into .csv  (dict -> df -> .csv)
    # avgs_outcomes = pd.DataFrame(avgs_outcomes)
    # out_filename = f"outcomes_{bufn.name}.csv"
    # path = os.path.join(os.getcwd(), "data", out_filename)
    # avgs_outcomes.to_csv(path)
    #
    # exp_filename = f"experiments_{bufn.name}.csv"
    # path = os.path.join(os.getcwd(), "data", exp_filename)
    # experiments.to_csv(path)

ReadError: not a gzip file

In [ ]:
# deffuant_out = outcomes_dict[BeliefUpdate.DEFFUANT.name]
#
# deffuant_out


# # Dropping unwanted column
# results = results.rename(columns={"Unnamed: 0": "policy"})
# results


In [ ]:
# # rescale lever values (0-10 -> 0-100, 0-5 -> 0-50)
# lever_names = get_lever_names()
# for col_name in results.columns:
#     if col_name in lever_names:
#         results[col_name] *= 10
#
# results

In [ ]:
# data_dict = {BeliefUpdate.DEFFUANT.name: results}